In [1]:
import findspark
import pyspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('spark://172.27.80.1:7077').appName('duplicate').getOrCreate()

In [3]:
dataF = spark.createDataFrame([('1','2400','32','9','3','PHILIPS','1XZFG')
                            , ('2','2400','23','13','3','Apple','2XZFG')
                            , ('3','2400','22','2.5','0','LG','3XZFG')
                            , ('4','2400','33','16.5','3','None','4XZFG')
                            , ('8','3500','33','23','5.5','LG','8XZFG')
                            , ('6','2400','23','9','0','SONY','6XZFG')
                            , ('7','3500','33','23','5.5','PHILIPS','7XZFG')
                            , ('8','3500','33','23','5.5','LG','8XZFG')
                            , ('9','2400','22','2.5','0','Apple','9XZFG')
                            , ('18','2400','22','2.5','0','Apple','9XZFG')
                            , ('10','2400','23','6','0','None','10XZFG')
                            , ('15','2450','23','8','1','SONY','DCEZX15')
                            , ('11','1200','23','6','2','SONY','11XZFG')
                            , ('12','3500','23','6','2','LG','12XZFG')
                            , ('17','3500','23','6','2','LG','12XZFG')
                            , ('13','2400','33','16.5','5.5','Apple','13XZFG')
                            , ('14','3500','23','6','0','DELL','14XZFG')
                            , ('15','2400','22','6','1','SONY','15XZFG')],
                              ['Id','Salary','age','experience','Coef','TV ','CODE_ID'])

In [5]:
#중복값 확인
dataF.count(), dataF.distinct().count()

(18, 17)

In [6]:
all_deleted = dataF.dropDuplicates()

In [7]:
(dataF.groupBy(dataF.columns).count().filter('count > 1').show())

+---+------+---+----------+----+---+-------+-----+
| Id|Salary|age|experience|Coef|TV |CODE_ID|count|
+---+------+---+----------+----+---+-------+-----+
|  8|  3500| 33|        23| 5.5| LG|  8XZFG|    2|
+---+------+---+----------+----+---+-------+-----+



In [8]:
#col중 id가 아닌 것들 리턴 -> 그중 2건씩 나오는 것들
(all_deleted.groupBy([col for col in all_deleted.columns if col != 'Id']).count().filter('count > 1').show())

+------+---+----------+----+-----+-------+-----+
|Salary|age|experience|Coef|  TV |CODE_ID|count|
+------+---+----------+----+-----+-------+-----+
|  2400| 22|       2.5|   0|Apple|  9XZFG|    2|
|  3500| 23|         6|   2|   LG| 12XZFG|    2|
+------+---+----------+----+-----+-------+-----+



In [9]:
#subset조건에 맞는 col 리턴
# id col빼고 중복되는값 제외 리턴
id_deleted = all_deleted.dropDuplicates(subset=[col for col in all_deleted.columns if col != 'Id'])

In [10]:
#같아지면 중복된거 해결
id_deleted.count(), id_deleted.distinct().count()

(15, 15)

[agg로 중복값 처리]

In [12]:
import pyspark.sql.functions as sqlfunct

In [13]:
id_deleted.agg(
    sqlfunct.count('Id').alias('Count_Ids'),
    sqlfunct.countDistinct('Id').alias('Count_Distinct_Ids')
).show()
# 전체 count / 중복 제거 count -> 중복값이 1개 있다 확인

+---------+------------------+
|Count_Ids|Count_Distinct_Ids|
+---------+------------------+
|       15|                14|
+---------+------------------+



In [14]:
#중복된 값 확인
(id_deleted.groupBy('Id').count().filter('count > 1').show())

+---+-----+
| Id|count|
+---+-----+
| 15|    2|
+---+-----+



In [15]:
#빅데이터에선 spark 내장 함수를 안 쓰면 중복값 생길 수 있음
new_id = (
    id_deleted.select(
    [sqlfunct.monotonically_increasing_id().alias('Id')]+
    [col for col in id_deleted.columns if col != 'Id']
    )
)

In [16]:
new_id.show()

+-------------+------+---+----------+----+-------+-------+
|           Id|Salary|age|experience|Coef|    TV |CODE_ID|
+-------------+------+---+----------+----+-------+-------+
|  25769803776|  3500| 23|         6|   2|     LG| 12XZFG|
|  51539607552|  3500| 33|        23| 5.5|PHILIPS|  7XZFG|
| 249108103168|  2400| 32|         9|   3|PHILIPS|  1XZFG|
| 412316860416|  2400| 22|       2.5|   0|  Apple|  9XZFG|
| 498216206336|  2400| 23|         6|   0|   None| 10XZFG|
| 566935683072|  2400| 23|        13|   3|  Apple|  2XZFG|
| 704374636544|  1200| 23|         6|   2|   SONY| 11XZFG|
| 721554505728|  2400| 33|      16.5|   3|   None|  4XZFG|
| 893353197568|  3500| 23|         6|   0|   DELL| 14XZFG|
| 962072674304|  2400| 33|      16.5| 5.5|  Apple| 13XZFG|
| 970662608896|  2400| 22|         6|   1|   SONY| 15XZFG|
|1133871366144|  3500| 33|        23| 5.5|     LG|  8XZFG|
|1142461300736|  2400| 23|         9|   0|   SONY|  6XZFG|
|1503238553600|  2400| 22|       2.5|   0|     LG|  3XZF